# Lab 1: Tokenisation and embeddings

In this lab, you will build an understanding of how text can be transformed into representations that computers can process and learn from. Specifically, you will explore two key concepts: *tokenisation* and *embeddings*. Tokenisation splits text into smaller units such as words, subwords, or characters. Embeddings are dense, fixed-size vector representations of tokens in a continuous space.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

## Part 1: Tokenisation

In the first part of the lab, you will code and analyse a tokeniser based on the Byte Pair Encoding (BPE) algorithm.

### Utility functions

The BPE tokeniser transforms text into a list of integers representing tokens. As a warm-up, you will implement two utility functions on such lists. To simplify things, we define a shorthand for the type of pairs of integers:

In [1]:
 Pair = tuple[int, int]

#### 🎈 Task 1.01: Counting pairs

Write a function that counts all occurrences of pairs of consecutive token IDs in a given list. The function should return a dictionary that maps each pair to its count. Skip counts that are zero.

In [2]:
def count(ids: list[int]) -> dict[Pair, int]:
    # TODO: Replace the following line with your own code
    tokenIds = {}
    for i in range(len(ids)-1):

        current_pair = (ids[i], ids[i+1])
        tokenIds[current_pair] = tokenIds.get(current_pair, 0) + 1
    #print(tokenIds)
    return tokenIds
ids = [0,1,2,3,5,3,5]
count(ids)

{(0, 1): 1, (1, 2): 1, (2, 3): 1, (3, 5): 2, (5, 3): 1}

#### 🎈 Task 1.02: Replacing pairs

Write a function that replaces all occurrences of a specified pair of consecutive token IDs in a given list by a new ID. The function should return the modified list.

In [3]:
def replace(ids: list[int], pair: Pair, new_id: int) -> list[int]:
    # TODO: Replace the following line with your own code
    rmv = []
    for i in range(len(ids)-1):
        if (ids[i], ids[i + 1]) == pair:
            rmv.insert(0, i+1)
            ids[i] = new_id
    for i in rmv:
        ids.pop(i)        
    return ids

In [5]:
replace([0,1,2,3,5,1,2,5], (2,5), 99)

[0, 1, 2, 3, 5, 1, 99]

### Encoding and decoding

The next cell contains the core code for the tokeniser in the form of a class `Tokenizer`. This class implements two methods: `encode()` converts an input text to a list of token IDs by exhaustively applying rules for merging pairs of consecutive IDs, and `decode()` reverses this process by looking up the tokens corresponding to the token IDs.

In [ ]:
class Tokenizer:
    def __init__(self):
        self.merges = {} #Dictionary/Mapping of tokens to merge next [Pair, int]
        self.vocab = {i: bytes([i]) for i in range(2**8)} #[int, bytes]

    def encode(self, text):
        ids = list(text.encode("utf-8"))
        while True:
            counts = count(ids)
            mergeable_pairs = counts.keys() & self.merges.keys()
            if len(mergeable_pairs) == 0:
                break
            to_merge = min(mergeable_pairs, key=self.merges.get)
            #print(f"to mrege : {to_merge}")
            ids = replace(ids, to_merge, self.merges[to_merge])
        return ids

    def decode(self, ids):
        return b"".join((self.vocab[i] for i in ids)).decode("utf-8")

#### 🎓 Task 1.03: Encoding and decoding

Explain how the code implements the BPE algorithm. Use the following steps to check your understanding:

**Step&nbsp;1.** Annotate the attributes and methods of the `Tokenizer` class with their Python types. In particular, what is the type of `self.merges`? Use the `Pair` shorthand.

**Step&nbsp;2.** Explain how the implementation chooses which merge rule to apply. Provide an example that illustrates the logic.

Answer

The implementation chooses the next pair in self.merges to merge. 

Assuming we have the following in self.merges: {(65,66):1, (66,67):1} and the input "ABC" is igven, it will first merge "AB" and then "C", giving us the final encoded values of [256, 67].  

### Training a tokeniser

Upon initialisation, a tokeniser has an empty set of merge rules. Your next task is to complete the BPE algorithm and write code to learn these merge rules from a text.

#### 🎓 Task 1.04: Training a tokeniser

Write a function that induces a BPE tokeniser from a given text. The function should take the text (a string) and a target vocabulary size as input and return the trained tokeniser.

In [6]:

def from_text(text: str, vocab_size: int) :
    # TODO: Replace the following line with your own code
    tokenizer = Tokenizer()
    ids = list(text.encode("utf-8"))
    com_val = len(tokenizer.vocab)
    while len(tokenizer.vocab) < vocab_size:
        max_key = max(count(ids), key=count(ids).get)
        com_val = com_val + 1
        tokenizer.merges[max_key] = len(tokenizer.vocab)
        tokenizer.vocab[len(tokenizer.vocab)] = com_val
        ids =  replace(ids, max_key, tokenizer.merges[max_key])
    return tokenizer

In [7]:
def read_file(filename: str):
    with open(filename, "r", encoding="utf-8") as f:
        return f.read()


To help you test your implementation, we provide three text files together with tokenisers trained on these files. Each text file contains the first 1&nbsp;million Unicode characters in a language-specific Wikipedia:

| Text file | Tokeniser file | Wikipedia |
|---|---|---|
| `wiki-en-1m.txt` | `wiki-en-1m.tok` | [Simple English](https://simple.wikipedia.org/) |
| `wiki-is-1m.txt` | `wiki-is-1m.tok` | [Icelandic](https://is.wikipedia.org/) |
| `wiki-sv-1m.txt` | `wiki-sv-1m.tok` | [Swedish](https://sv.wikipedia.org/) |

A tokeniser file consists of lines specifying merge rules. For example, the first line in the tokeniser file for Swedish is `101 114`, which expresses that this rule combines the token with ID 101 (`e`) and the token with ID 114 (`r`). The ID of the new token (`er`) is 256 plus the (zero-indexed) line number on which the rule is found. The following code saves a `Tokenizer` to a file with this format:

In [8]:
def save(tokenizer: Tokenizer, filename: str) -> None:
    with open(filename, "w") as f:
        for fst, snd in tokenizer.merges:
            print(f"{fst} {snd}", file=f)

In [9]:
text = read_file(filename="wiki-en-1m.txt")
tokenizer = from_text(text, 1024)
save(tokenizer, "our_tokenizer.tok")

To test your code, compare the saved tokeniser to the provided tokeniser using `diff`.

### Tokenisation quirks

The tokeniser is a key component of language models, as it defines the minimal chunks of text the model can “see” and work with. As you will see in this section, tokenisation is also responsible for several deficiencies and unexpected behaviours of language models.

One helpful tool for experimenting with tokenisers in language models is the web app [Tiktokenizer](https://tiktokenizer.vercel.app/). This app lets you play around with, among others, [`cl100k_base`](https://tiktokenizer.vercel.app/?model=cl100k_base), the tokeniser used in the free version of ChatGPT and OpenAI’s APIs, and [`o200k_base`](https://tiktokenizer.vercel.app/?model=o200k_base), used in GPT-4o.

#### 🎓 Task 1.05: Tokenisation quirks

Prompt [ChatGPT](https://chatgpt.com/) to reverse the letters in the following words:

```
creativecommons
MERCHANTABILITY
NSNotification
authentication
```

How many of these words come out right? What could be the problem when words come out wrong? Generate ideas by inspecting the words in Tiktokenizer. Try to come up with other prompts that illustrate problems related to tokenisation.

Answer

We had various results from GPT. One time it managed to only do authentication, once it did not manage to revert a single one and once it managed to revert every single word perfectly. The problem exists in the words being 1 token, since GPT  does not know which letters are inside each token, it becomes difficult to revert the word. If the word consits of several smaller tokens it becomse easier for the model to revert it. For example if each letter is 1 token, it can easily revert it since a reverted is just a.  

### Tokenisation and multi-linguality

Many NLP systems and the tokenisers used with them are primarily trained on English data. In the next task, you will reflect on the effect this has when they are used to process non-English data.

The *context length* of a language model is the maximum number of preceding tokens the model can condition on when predicting the next token. This number is fixed and cannot be changed after training the model. For example, the context length of GPT-2 ([Radford et al., 2019](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)) is 1,024. 

While the context length of a language model is fixed, the amount of information that can be squeezed into this context length will depend on the tokeniser. Informally speaking, a model that needs more tokens to represent a given text cannot condition on as much information as one that needs fewer tokens.

#### 🎓 Task 1.06: Tokenisation and multi-linguality

Train a tokeniser on the English text file from Task&nbsp;1.04 and test it on the same text. How many tokens does it split the text into? Based on this, what is the expected number of Unicode characters of English text that can be fit into a context length of 1,024?

What do the numbers look like if you test the English tokeniser on the Icelandic text instead? What could explain the differences?

Interpreting the expected number of Unicode characters as a measure of representation efficiency, what do your results tell you about the efficiency of a language model primarily trained on English data when it is used to process non-English data? Why are these findings relevant?

In [10]:
with open('wiki-en-1m.txt', 'r') as file:
    english_text = file.read()

with open('wiki-is-1m.txt', 'r') as file:
    icelandic_text = file.read()

english_tokens = tokenizer.encode(english_text)
icelandic_tokens = tokenizer.encode(icelandic_text)

# Calculate efficiency metrics
english_token_count = len(english_tokens)
icelandic_token_count = len(icelandic_tokens)

# Estimate how many Unicode characters fit into a context length of 1024
avg_english_chars_per_token = len(english_text) / english_token_count
avg_icelandic_chars_per_token = len(icelandic_text) / icelandic_token_count

expected_english_chars = avg_english_chars_per_token * 1024
expected_icelandic_chars = avg_icelandic_chars_per_token * 1024

# Display results
print("English token count:", english_token_count)
print("Icelandic token count:", icelandic_token_count)
print("Avg English chars per token:", avg_english_chars_per_token)
print("Avg Icelandic chars per token:", avg_icelandic_chars_per_token)
print("Expected English chars in 1024 context:", expected_english_chars)
print("Expected Icelandic chars in 1024 context:", expected_icelandic_chars)

English token count: 379524
Icelandic token count: 754575
Avg English chars per token: 2.6348794806125566
Avg Icelandic chars per token: 1.3252493125269191
Expected English chars in 1024 context: 2698.116588147258
Expected Icelandic chars in 1024 context: 1357.0552960275652


Answer

A language model trained on only English text loses efficiency when applied to non-English languages because it requires more tokens to represent the same amount of text. This leads decreased performance in terms of loss of context and strucure. These results are important because it shows the need for multilingual training data and tokenizer adaptation when building NLP systems for diverse languages.

## Part 2: Embeddings

In the second part of the lab, you will explore embeddings. An embedding layer is a network component that assigns each item in a finite set of elements (often called a *vocabulary*) a fixed-size vector. At first, these vectors are filled with random values, but during training, they are adjusted to suit the task at hand.

### Bag-of-words classifier

To help you build an intuition for embeddings and the vector representations learned by them, we will use a simple bag-of-words text classifier. The core part of this classifier only takes a few lines of code:

In [31]:
import torch.nn as nn


class Classifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.linear = nn.Linear(embedding_dim, num_classes)

        #task 1.12
        nn.init.kaiming_uniform_(self.embedding.weight, mode="fan_in", nonlinearity="linear")

    def forward(self, x):   
        return self.linear(self.embedding(x).mean(dim=-2))

#### 🎈 Task 1.07: Bag-of-words classifier

Explain how the bag-of-words classifier works. How does the code match the diagram you saw in the lectures? Why is there only one `nn.Embedding`, while the diagram shows three embedding layers? What does the keyword argument `dim=-2` do?

Answer

The code implements the different layers in the diagram, embedding and linear. There is only need for one layer because of the parameter "num_embeddings". 

Embedding layer: 
 - Input: A batch of tokenized text sequence 
 - Output: A tensor of shape (batch_size, sequence_length, embedding_dim)

Pooling layer:
 - Output: (batch_size, embedding_dim); where each sequence is now represented by a single averaged embedding.

Linear layer:
- Input: (batch_size, embedding_dim)
- Output: (batch_size, num_classes); representing class scores for each input sequence.

Given a tensor of shape (batch_size, sequence_length, embedding_dim)Given a tensor of shape (batch_size, sequence_length, embedding_dim), averaging over, for example, dim=-1 would output a tensor of shape (batch_size, sequence_length). We use dim=-2 to compute the mean across the sequence length dimension, reducing the sequence to a single vector per input.

### Dataset

You will apply the classifier to a small dataset with Amazon customer reviews. This dataset is taken from [a much larger dataset](https://www.cs.jhu.edu/~mdredze/datasets/sentiment/) first described by [Blitzer et al. (2007)](https://aclanthology.org/P07-1056/).

The dataset contains whitespace-tokenised product reviews from two topics: cameras (`camera`) and music (`music`). Each review is additionally annotated for sentiment towards the product at hand: negative (`neg`) or positive (`pos`). The topic and sentiment labels are prepended to the review. As an example, here is the first review from the training data:

```
music neg oh man , this sucks really bad . good thing nu-metal is dead . thrash metal is real metal , this is for posers
```

The next cell contains a custom [`Dataset`](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) class for the review dataset. To initialise an instance of this class, you specify the name of the file containing the reviews you want to load (`filename`) and which of the two labels you want to use (`label`): topic (0) or sentiment (1).

In [17]:
from torch.utils.data import Dataset


class ReviewDataset(Dataset):
    def __init__(self, filename: str, label: int = 0) -> None:
        with open(filename) as f:
            tokenized_lines = [line.split() for line in f]
        self.items = [(tokens[2:], tokens[label]) for tokens in tokenized_lines]

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, idx: int) -> tuple[list[str], str]:
        return self.items[idx]

data = ReviewDataset("reviews-train.txt", 0)

### Vectoriser

To feed a review into the bag-of-words classifier, you first need to turn it into a vector of token IDs. Likewise, you need to convert the label (topic or sentiment) into an integer. The next cell contains a partially completed `ReviewVectoriser` class that handles this transformation.

In [18]:
from collections import Counter

import torch

# Type abbreviation for review–label pairs
Item = tuple[list[str], str]


class ReviewVectorizer:
    PAD = "[PAD]"
    UNK = "[UNK]"

    def __init__(self, dataset: ReviewDataset, n_vocab: int = 1024) -> None:
        # Unzip the dataset into reviews and labels
        reviews, labels = zip(*dataset)

        # Count the tokens and get the most common ones
        counter = Counter(t for r in reviews for t in r)
        most_common = [t for t, _ in counter.most_common(n_vocab - 2)]
        # Create the token-to-index and label-to-index mappings
        self.t2i = {t: i for i, t in enumerate([self.PAD, self.UNK] + most_common)}
        self.l2i = {l: i for i, l in enumerate(sorted(set(labels)))}
    def __call__(self, items: list[Item]) -> tuple[torch.Tensor, torch.Tensor]:
        # Create empty lists for the tokenized reviews and label IDs
        tokenized_reviews = []
        label_ids = []
        
        # Find the length of the longest review for padding
        max_len = max(len(review) for review, _ in items)
        
        # Loop over the review-label pairs
        for review, label in items:
            # Convert review to token IDs, replacing unknown tokens with UNK
            token_ids = [self.t2i.get(t, self.t2i[self.UNK]) for t in review]
            
            # Pad with PAD token IDs to match max_len
            token_ids.extend([self.t2i[self.PAD]] * (max_len - len(token_ids)))
            
            tokenized_reviews.append(token_ids)
            label_ids.append(self.l2i[label])
            
        # Convert to PyTorch tensors
        X = torch.tensor(tokenized_reviews, dtype=torch.long)
        y = torch.tensor(label_ids, dtype=torch.long)
        
        return X, y
    
ReviewVectorizer(data, 1024)

A `ReviewVectoriser` maps tokens and labels to IDs using two Python dictionaries. These dictionaries are set up when the vectoriser is initialised and queried when the vectoriser is called on a batch of review–label pairs. They include IDs for two special tokens:

`[PAD]` (Padding): Reviews can have different lengths, but PyTorch requires all vectors in a batch to be the same size. To handle this, the vectoriser adds `[PAD]` tokens to the end of shorter reviews so they match the length of the longest review in the batch.

`[UNK]` (Unknown): If a review contains a token that is not in the token-to-ID dictionary, the vectoriser assigns it the ID of the `[UNK]` token instead of a regular ID.

#### 🎓 Task 1.08: Vectoriser

Explain and complete the code of the vectoriser. Follow these steps:

**Step&nbsp;1.** Explain how unzipping works. What are the types of `reviews` and `labels`?

**Step&nbsp;2.** Explain how the token-to-ID and label-to-ID mappings are constructed. How does the `most_common()` method deal with elements that occur equally often?

**Step&nbsp;3.** Complete the implementation of the `__call__()` method. This method should convert a list of $m$ review–label pairs into a pair $(X, y)$ where $X$ is a matrix containing the vectors with token IDs for the reviews, and $y$ is a vector containing the IDs of the corresponding labels.

1.  Reviews is a tuple containing lists of strings. Each list contains all the words for 1 review.

    Labels are a tuple of all labels/sentiments, e.g ('music', 'camera', 'camera'... etc)/('neg', 'pos', 'pos'... etc)

    unzip splits pairs of (review, label) into two separate tuples
    
    
2.  t2i enumerates the most_common. It adds the PAD and UNK in the beginning since they are removed from most previous (The -2 in the for loop.)

    l2i removes all duplicates and assigns a unique id to each unique sentiment or label.

    most_common() deals with it by taking the element that appears first. 



### Training the classifier

With the vectoriser completed, you are ready to train a classifier. More specifically, you can train two separate classifiers: one to predict the topic of a review, and one to predict the sentiment. The next cell contains a simple training loop that you can adapt for this purpose.

In [ ]:
import torch.nn.functional as F


def train(filename = "reviews-train.txt", label = 0, lr = 0.001, batch_size=16, shuffle=True, epoch_count = 10):
    #Loads the required variables needed for training, with their respective variables. 
    dataset = ReviewDataset(filename, label)
    processor = ReviewVectorizer(dataset, 1024)
    model = Classifier(1024, 64, len(processor.l2i))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=processor,
    )
    #Training loop for x amount of epochs
    for epoch in range(epoch_count):
        model.train() #Sets model to training mode
        running_loss = 0 #init loss for this epoch 
        for bx, by in data_loader:
            optimizer.zero_grad() #Make sure no previous gradients
            output = model(bx) #Model forward pass
            loss = F.cross_entropy(output, by) #Calculate loss (Cross entropy in this case)
            loss.backward() #Backward pass
            optimizer.step() #Update model parameters from the optimizer
            running_loss += loss.item() #Accumulte loss
        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    return processor, model

#### 🎓 Task 1.09: Training loop

Explain the training loop. Follow these steps:

**Step&nbsp;1.** Go through the training loop line-by-line and add comments where you find it suitable. Your comments should be detailed enough for you to explain the main steps of the loop.

**Step&nbsp;2.** The training loop contains various hard-coded values like filename, learning rate, batch size, and epoch count. This makes the code less flexible. Revise the code so that you can specify these values using keyword arguments. Use the concrete values from the code as defaults.

#### 🎈 Task 1.10: Training the classifier

Adapt the next cell to train the classifier for the two prediction tasks. Based on the loss values, which task appears to be the harder one? What is the purpose of setting a seed?

In [32]:
torch.manual_seed(42)
vectorizer_0, model_0 = train()
vectorizer_1, model_1 = train(label=1)

Epoch 0, loss: 0.6797
Epoch 1, loss: 0.6109
Epoch 2, loss: 0.4807
Epoch 3, loss: 0.3550
Epoch 4, loss: 0.2697
Epoch 5, loss: 0.2153
Epoch 6, loss: 0.1790
Epoch 7, loss: 0.1500
Epoch 8, loss: 0.1316
Epoch 9, loss: 0.1146
Epoch 0, loss: 0.6908
Epoch 1, loss: 0.6839
Epoch 2, loss: 0.6675
Epoch 3, loss: 0.6353
Epoch 4, loss: 0.5904
Epoch 5, loss: 0.5457
Epoch 6, loss: 0.5043
Epoch 7, loss: 0.4651
Epoch 8, loss: 0.4364
Epoch 9, loss: 0.4092


Based on the loss, label=1 (sentiment) seems to be harder to train, as the loss decreases slower. 

The purpose of setting a seed is to ensure the trained model behaves the same way each time you train it, allowing you to reproduce your previous results.

### Inspecting the embeddings

Now that you have trained the classifier on two separate prediction tasks, it is interesting to inspect and compare the embedding vectors it learned in the process. For this you will use an online tool called the [Embedding Projector](http://projector.tensorflow.org). The next cell contains code to save the embeddings from a trained classifier in a format that can be loaded into this tool.

In [20]:
def save_embeddings(
    vectorizer: ReviewVectorizer,
    model: Classifier,
    vectors_filename: str,
    metadata_filename: str,
):
    i2t = {i: t for t, i in vectorizer.t2i.items()}
    embeddings = model.embedding.weight.detach().numpy()
    items = [(i2t[i], e) for i, e in enumerate(embeddings)]
    with open(vectors_filename, "wt") as f1, open(metadata_filename, "wt") as f2:
        for w, e in items:
            print("\t".join("{:.5f}".format(x) for x in e), file=f1)
            print(w, file=f2)

Call this code as follows:

In [27]:
save_embeddings(vectorizer_1, model_1, "vectors_1.tsv", "metadata_1.tsv")
save_embeddings(vectorizer_0, model_0, "vectors_0.tsv", "metadata_0.tsv")

#### 🎓 Task 1.11: Inspecting the embeddings

Load the embeddings from the two classification tasks (topic classification and sentiment classification) into the Embedding Projector web app and inspect the vector spaces. How do they compare visually? Does the visualisation make sense to you?

The Embedding Projector offers visualisations based on three dimensionality reduction methods: [UMAP](https://umap-learn.readthedocs.io/en/latest/), [T-SNE](https://en.m.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding), and [PCA](https://en.m.wikipedia.org/wiki/Principal_component_analysis). Which of these seems most useful to you?

Focus on the embeddings for the words *repair* and *sturdy*. Are they close to each other or far away from another? What happens if you switch to the other task? How do you explain that?

Answer:

The visualization makes sense, with T-SNE being a bit more confusing than the other. Words with similar meaning are closer together. X seems most useful. 

UMAP: They are relatively close compared to other points, but they are not the closest. They are also not in the same cube.

T-SNE:  In this one they seem closer together, being in the same area of the space.

PCA: Finally in PCA it was the most intuitive visualization, once again the two words being in the same quadrant.

### Initialisation of embedding layers

The error surfaces explored when training neural networks can be very complex. Because of this, it is crucial to choose “good” initial values for the parameters. In the final task of this lab, you will run a small experiment to see how alternative initialisations can affect a model’s performance.

In PyTorch, the weights of the embedding layer are initially set by sampling from the standard normal distribution, $\mathcal{N}(0, 1)$. However, research suggests other approaches may work better. For example, you have seen that embedding layers share similarities with linear layers, so it makes sense to use the same initialisation method for both. The default initialisation method for linear layers in PyTorch is the so-called Kaiming initialisation, introduced by [He et al. (2015)](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf).

#### 🎈 Task 1.12: Initialisation of embedding layers

Check the [source code of `nn.Linear`](https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear) to see how PyTorch initialises the weights of linear layers using the Kaiming initialisation method. Apply the same method to the embedding layer of your classifier and see how this affects the loss of your model and the vector spaces.

Answer

The loss decreases faster and reaches a lower value. 
We were not able to see how it affected the model in the vectorspaces.

**🥳 Congratulations on finishing lab&nbsp;1!**